In [1]:
# Setup environment
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/Othercomputers/My_Mac/sentinel')

import os
os.chdir('/content/drive/Othercomputers/My_Mac/sentinel')

# Install dependencies
!pip install -q torch torchvision open3d google-cloud-storage numpy tqdm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.4 MB/s eta 0:00:00


In [8]:
import sys
sys.path.append("/content/drive/Othercomputers/My_Mac/sentinel")


In [10]:
!touch /content/drive/Othercomputers/My_Mac/sentinel/src/__init__.py
!touch /content/drive/Othercomputers/My_Mac/sentinel/src/python/__init__.py
!touch /content/drive/Othercomputers/My_Mac/sentinel/src/python/utils/__init__.py


In [12]:
from src.python.utils.visualization import visualize_pointcloud


In [13]:
from src.python.datasets.semantic_kitti import SemanticKITTIDataset

import numpy as np
import torch
from torch.utils.data import DataLoader

In [16]:
from google.colab import auth
auth.authenticate_user()
# Test dataset loading from GCS
dataset = SemanticKITTIDataset(
    root_dir='',  # Not used with GCS
    split='train',
    use_gcs=True,
    bucket_name='sentinel-kitti-data'
)

print(f"Dataset size: {len(dataset)} samples")

# Load a sample
points, labels = dataset[0]
print(f"Points shape: {points.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Unique labels: {torch.unique(labels)}")


Dataset size: 19130 samples
Points shape: torch.Size([50000, 3])
Labels shape: torch.Size([50000])
Unique labels: tensor([ 0,  1,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19])


In [22]:
# Data augmentation functions
import torch
import numpy as np

class PointCloudAugmentation:
    def __init__(self, config):
        self.config = config

    def random_rotate(self, points):
        angle = np.random.uniform(0, 2 * np.pi)
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        rotation_matrix = np.array([
            [cos_angle, -sin_angle, 0],
            [sin_angle, cos_angle, 0],
            [0, 0, 1]
        ])
        points_np = points.numpy()
        points_np = points_np @ rotation_matrix.T
        return torch.from_numpy(points_np).float()

    def random_scale(self, points):
        scale = np.random.uniform(0.9, 1.1)
        return points * scale

    def random_jitter(self, points):
        noise = torch.randn_like(points) * 0.01
        return points + noise

    def random_dropout(self, points, labels):
        keep_ratio = np.random.uniform(0.8, 1.0)
        num_points = points.shape[0]
        num_keep = int(num_points * keep_ratio)
        indices = torch.randperm(num_points)[:num_keep]
        return points[indices], labels[indices]

    def __call__(self, points, labels):
        if self.config.get('random_rotate', False):
            points = self.random_rotate(points)
        if self.config.get('random_scale', False):
            points = self.random_scale(points)
        if self.config.get('random_jitter', False):
            points = self.random_jitter(points)
        if self.config.get('random_dropout', False):
            points, labels = self.random_dropout(points, labels)
        return points, labels


In [23]:
# Test augmentation
aug_config = {
    'random_rotate': True,
    'random_scale': True,
    'random_jitter': True,
    'random_dropout': True
}

augmentation = PointCloudAugmentation(aug_config)
aug_points, aug_labels = augmentation(points, labels)

print(f"Augmented points shape: {aug_points.shape}")
print(f"Augmented labels shape: {aug_labels.shape}")


Augmented points shape: torch.Size([40517, 3])
Augmented labels shape: torch.Size([40517])


In [24]:
# Save augmentation module
augmentation_code = '''
import torch
import numpy as np

class PointCloudAugmentation:
    # ... (same as above)
'''

# Write to file
with open('src/python/datasets/data_augmentation.py', 'w') as f:
    f.write(augmentation_code)

print("Data augmentation module saved!")


Data augmentation module saved!
